# Graph template

## Problem 743. Network Delay Time
You are given a network of n nodes, labeled from 1 to n. You are also given times, a list of travel times as directed edges times[i] = (ui, vi, wi), where ui is the source node, vi is the target node, and wi is the time it takes for a signal to travel from source to target.

We will send a signal from a given node k. Return the time it takes for all the n nodes to receive the signal. If it is impossible for all the n nodes to receive the signal, return -1.

### Example output
Example 1:

```
Input: times = [[2,1,1],[2,3,1],[3,4,1]], n = 4, k = 2
Output: 2
Example 2:

Input: times = [[1,2,1]], n = 2, k = 1
Output: 1
Example 3:

Input: times = [[1,2,1]], n = 2, k = 2
Output: -1
```

### Constraints:

```
1 <= k <= n <= 100
1 <= times.length <= 6000
times[i].length == 3
1 <= ui, vi <= n
ui != vi
0 <= wi <= 100
All the pairs (ui, vi) are unique. (i.e., no multiple edges.)
```

## generate input

In [211]:
import numpy as np
def gen(N,M,W,Size):
    for _ in range(Size):
        n = np.random.randint(3,N)
        k = np.random.randint(1,n)
        m = min(np.random.randint(1,M),n*(n-1))
        uvpool = [(i,j) for i in range(1,n+1) for j in range(1,n+1) if i!=j]
        uvw = []
        for _ in range(m):
            # print(len(uvpool),m,len(uvw),n,n*(n-1))
            i = np.random.randint(0,len(uvpool)-1) if len(uvpool)>1 else 0
            uvw.append((*uvpool[i],np.random.randint(0,W)))
            uvpool.pop(i)
        yield uvw,n,k

for uvw,n,k in gen(10,1000,100,2):
    print(uvw,n,k)

[(1, 2, 82), (2, 1, 1), (2, 3, 30), (3, 1, 89), (1, 3, 51), (3, 2, 10)] 3 1
[(3, 1, 12), (1, 3, 42), (2, 3, 45), (2, 1, 22), (1, 2, 79), (3, 2, 72)] 3 2


## two ways of storing edges
### When $M \sim N^2$

In [20]:
n = 100

INF = 0x3f3f3f
w=[[INF]*n for _ in range(n)]
def add(a,b,c):
    w[a][b] = c
    
def visit(a):
    for b in range(n):
        print(a,b,w[a][b])

### When $M\sim N$

In [21]:
n = 100
m = 200

EMPTY = -1
he = [EMPTY]*n # no edges
ne = [None]*m
e = [None]*m
w = [None]*m
idx = 0

def add2(a,b,c):
    ne[idx] = he[a]
    he[a] = idx
    e[idx] = b
    w[idx] = c
    idx += 1

def visit2(a):
    i = he[a]
    while i!=EMPTY:
        print(a,e[i],w[i])
        i = ne[i]

## Floyd + adjacency matrix

In [168]:
class FloydAdjMatrix:
    INF = 0x3f3f3f3f
    def run(self,ts,n,k):
        w = [[INF]*(n+1) for _ in range(n+1)]
        for i in range(1,n+1):
            w[i][i] = 0
        
        for u,v,c in ts:
            w[u][v] = min(w[u][v],c)
            
        self.floyd(w,n)
        
        ans = max(w[k][1:])
        
        return -1 if ans>=INF/2 else ans
    
    def floyd(self,w,n):
        for p in range(1,n+1):
            for i in range(1,n+1):
                for j in range(1,n+1):
                    w[i][j] = min(w[i][j],w[i][p]+w[p][j])

In [169]:
class DijkstraAdjMatrix:
    INF = 0x3f3f3f3f
    def run(self,ts,n,k):
        w = [[INF]*(n+1) for _ in range(n+1)]
        for i in range(1,n+1):
            w[i][i] = 0
        
        for u,v,c in ts:
            w[u][v] = min(w[u][v],c)
            
        dist = self.dijkstra(w,n,k)
        
        ans = max(dist[1:])
            
        return -1 if ans>=INF/2 else ans
    
    def dijkstra(self,w,n,k):
        vis = [False]*(n+1)
        dist = [INF]*(n+1)
        dist[k] = 0
        
        for p in range(1,n+1):
            # find index of min distance
            t = -1
            for i in range(1,n+1):
                if not vis[i] and (t==-1 or dist[i]<dist[t]):
                    t = i
            vis[t] = True # we will not update it anymore
            
            for i in range(1,n+1):
                if not vis[i]:
                    dist[i] = min(dist[i],dist[t] + w[t][i])
        
        return dist

In [248]:
from collections import *
from heapq import *
class DijkstraAdjList:
    INF = 0x3f3f3f3f
    EMPTY = -1
    def run(self,ts,n,k):
        m = len(ts)
        he = [EMPTY]*(n+1) # head
        ne = [None]*m # next
        e = [None]*m # dst
        w = [None]*m # weight

        # self.G = defaultdict(list)
        for idx,(u,v,c) in enumerate(ts):
            ne[idx] = he[u] # point to previous head
            he[u] = idx # new head (will point to previous head)
            e[idx] = v
            w[idx] = c
            # self.G[u].append((v,c))
            
        dist = self.dijkstra(m,he,ne,e,w,n,k)
        
        ans = max(dist[1:])
        
        return -1 if ans>=INF/2 else ans
    
    def dijkstra(self,m,he,ne,e,w,n,k):
        dist = [INF]*(n+1)
        dist[k] = 0
        vis = [False]*(n+1)
        heap = [(0,k)]
        while heap:
            # print(dist,vis,heap,k)
            d,p = heappop(heap)
            if vis[p]:
                continue
            vis[p] = True
            
            idx = he[p]
            # print(f'G: {p}->{self.G[p]}')
            while idx != EMPTY: # idx = 0,...m-1 or None (empty)
                j = e[idx]
                if not vis[j] and d+w[idx]<dist[j]:
                    # print(f'{k}->({d})->{p}->({w[idx]})->{j}: {dist[j]}->{d+w[idx]}')
                    dist[j] = d+w[idx]
                    heappush(heap,(dist[j],j))
                idx = ne[idx]   
            # print(dist)
            # print(dist,vis,heap)
        return dist

In [216]:
class BellmanFord:
    INF = 0x3f3f3f3f
    def run(self,ts,n,k):
        dist = self.bf(ts,n,k)
        ans = max(dist[1:])
        return -1 if ans>=INF/2 else ans
    
    def bf(self,ts,n,k):
        dist = [INF]*(n+1)
        dist[k] = 0
        
        for _ in range(n):
            clone = dist[:]
            for u,v,c in ts:
                dist[v] = min(dist[v],clone[u]+c)
                
        return dist

In [259]:
class SPFAAdjList:
    INF = 0x3f3f3f3f
    EMPTY = -1
    def run(self,ts,n,k):
        m = len(ts)
        he = [EMPTY]*(n+1)
        ne = [None]*m
        e = [None]*m
        w = [None]*m
        for i,(u,v,c) in enumerate(ts):
            ne[i] = he[u]
            he[u] = i
            e[i] = v
            w[i] = c
            
        dist = self.spfa((he,ne,e,w),n,k)
        ans = max(dist[1:])
        return -1 if ans>=INF/2 else ans
    
    def spfa(self,adjlist,n,k):
        he,ne,e,w = adjlist
        dist = [INF]*(n+1)
        dist[k] = 0
        front = set([k])
        while front:
            # clone = dist[:]
            # newfront = set()
            newfront = front
            # for p in front:
            for p in [front.pop()]:
                # d = clone[p]
                d = dist[p]
                idx = he[p]
                while idx != EMPTY:
                    i = e[idx]
                    if d+w[idx]<dist[i]:
                        dist[i] = d+w[idx]
                        newfront.add(i)
                    idx = ne[idx]
                    # print(newfront,dist)
            front = newfront
        return dist

In [257]:
ts = [[4,2,76],[1,3,79],[3,1,81],[4,3,30],[2,1,47],[1,5,61],[1,4,99],[3,4,68],[3,5,46],[4,1,6],[5,4,7],[5,3,44],[4,5,19],[2,3,13],[3,2,18],[1,2,0],[5,1,25],[2,5,58],[2,4,77],[5,2,74]]
n = 5
k = 3
## Test
import time
sols = [FloydAdjMatrix,DijkstraAdjMatrix,DijkstraAdjList,SPFAAdjList]
ps = [0]*len(sols)
print(ts,n,k)
ans = []
for j,sol in enumerate(sols):
    t0 = time.time()
    ans.append(sol().run(ts,n,k))
    t1 = time.time()
    ps[j] += t1-t0
print(f'test {i:3d}: {ans}')
print(ps)

[[4, 2, 76], [1, 3, 79], [3, 1, 81], [4, 3, 30], [2, 1, 47], [1, 5, 61], [1, 4, 99], [3, 4, 68], [3, 5, 46], [4, 1, 6], [5, 4, 7], [5, 3, 44], [4, 5, 19], [2, 3, 13], [3, 2, 18], [1, 2, 0], [5, 1, 25], [2, 5, 58], [2, 4, 77], [5, 2, 74]] 5 3
{2} [4144959, 4144959, 18, 0, 4144959, 4144959]
{2, 5} [4144959, 4144959, 18, 0, 4144959, 46]
{2, 4, 5} [4144959, 4144959, 18, 0, 68, 46]
{1, 2, 4, 5} [4144959, 81, 18, 0, 68, 46]
{1, 2, 5} [4144959, 81, 18, 0, 68, 46]
{1, 2, 5} [4144959, 74, 18, 0, 68, 46]
{1, 2, 5} [4144959, 74, 18, 0, 68, 46]
{1, 2, 5} [4144959, 74, 18, 0, 68, 46]
{1, 2} [4144959, 74, 18, 0, 68, 46]
{1, 2} [4144959, 71, 18, 0, 68, 46]
{1, 2} [4144959, 71, 18, 0, 68, 46]
{1, 2, 4} [4144959, 71, 18, 0, 53, 46]
{1, 2} [4144959, 71, 18, 0, 53, 46]
{1, 2} [4144959, 59, 18, 0, 53, 46]
{1, 2} [4144959, 59, 18, 0, 53, 46]
{1, 2} [4144959, 59, 18, 0, 53, 46]
{2} [4144959, 59, 18, 0, 53, 46]
{2} [4144959, 59, 18, 0, 53, 46]
{2} [4144959, 59, 18, 0, 53, 46]
{2} [4144959, 59, 18, 0, 53, 46]

In [250]:
## Test
import time
sols = [FloydAdjMatrix,DijkstraAdjMatrix,DijkstraAdjList,SPFA]
ps = [0]*len(sols)
for i,(ts,n,k) in enumerate(gen(5,25,100,1)):
    print(ts,n,k)
    ans = []
    for j,sol in enumerate(sols):
        t0 = time.time()
        ans.append(sol().run(ts,n,k))
        t1 = time.time()
        ps[j] += t1-t0
    print(f'test {i:3d}: {ans}')
print(ps)

[(3, 1, 15), (1, 3, 27), (2, 1, 0), (1, 2, 1), (2, 3, 3), (3, 2, 69)] 3 1
test   0: [4, 4, 4, 4]
[2.9087066650390625e-05, 1.4066696166992188e-05, 1.3828277587890625e-05, 1.1920928955078125e-05]


In [260]:
## Test
import time
sols = [FloydAdjMatrix,DijkstraAdjMatrix,DijkstraAdjList,BellmanFord,SPFAAdjList]
ps = [0]*len(sols)
for i,(ts,n,k) in enumerate(gen(100,6000,1000,100)):
    ans = []
    for j,sol in enumerate(sols):
        t0 = time.time()
        ans.append(sol().run(ts,n,k))
        t1 = time.time()
        ps[j] += t1-t0
    print(f'test {i:3d}: {ans}')
print(ps)

test   0: [183, 183, 183, 183, 183]
test   1: [353, 353, 353, 353, 353]
test   2: [214, 214, 214, 214, 214]
test   3: [274, 274, 274, 274, 274]
test   4: [356, 356, 356, 356, 356]
test   5: [221, 221, 221, 221, 221]
test   6: [450, 450, 450, 450, 450]
test   7: [251, 251, 251, 251, 251]
test   8: [174, 174, 174, 174, 174]
test   9: [235, 235, 235, 235, 235]
test  10: [194, 194, 194, 194, 194]
test  11: [168, 168, 168, 168, 168]
test  12: [190, 190, 190, 190, 190]
test  13: [582, 582, 582, 582, 582]
test  14: [298, 298, 298, 298, 298]
test  15: [216, 216, 216, 216, 216]
test  16: [527, 527, 527, 527, 527]
test  17: [1597, 1597, 1597, 1597, 1597]
test  18: [173, 173, 173, 173, 173]
test  19: [308, 308, 308, 308, 308]
test  20: [129, 129, 129, 129, 129]
test  21: [162, 162, 162, 162, 162]
test  22: [275, 275, 275, 275, 275]
test  23: [171, 171, 171, 171, 171]
test  24: [169, 169, 169, 169, 169]
test  25: [302, 302, 302, 302, 302]
test  26: [211, 211, 211, 211, 211]
test  27: [352, 352, 35